<a href="https://colab.research.google.com/github/manjeet704/Datascience/blob/prac_code/Seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# link for dataset: http://www.manythings.org/anki/
!wget  http://www.manythings.org/anki/fra-eng.zip
!unzip -q fra-eng.zip -d /content/french-eng

--2024-02-14 16:37:56--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7833145 (7.5M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.47M  --.-KB/s    in 0.05s   

2024-02-14 16:37:56 (153 MB/s) - ‘fra-eng.zip’ saved [7833145/7833145]



In [2]:
#importing libraries
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [3]:
batch_s = 64
epochs = 100 # number of epochs for training
latent_dimension = 256 # # Latent dimensionality of the encoding space.
number_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = '/content/french-eng/fra.txt'

In [6]:
#Vectorize the data
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path, 'r', encoding='utf-8') as f:
  lines=f.read().split('\n')
for line in lines[: min(number_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [ ]:
input_texts

In [ ]:
target_texts

In [9]:
print("Number of input sample:",len(input_texts))
print("Number of input sample:",len(target_texts))

Number of input sample: 10000
Number of input sample: 10000


In [10]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 14
Max sequence length for outputs: 59


In [11]:
num_encoder_tokens

70

In [ ]:
input_characters

In [ ]:
target_characters

In [ ]:
num_decoder_tokens

In [ ]:
input_token_index = dict(
  [(char,i)  for i, char in enumerate(input_characters)])
input_token_index

In [ ]:
target_token_index = dict(
  [(char,i)  for i, char in enumerate(target_characters)])
target_token_index

## one hot encoding


In [17]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype ='float32')
decoder_input_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype ='float32')

decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype ='float32')

In [18]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [ ]:
decoder_target_data

In [20]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dimension, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [21]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dimension, return_sequences = True, return_state = True)
decoder_output,_, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_output)

In [22]:
model = Model([encoder_inputs, decoder_inputs],decoder_outputs )

In [23]:
# compile
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
              metrics =['accuracy'])

In [24]:
# training
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size = batch_s,
          epochs = 3,
          validation_split = 0.2)

Epoch 1/3
125/125 [==============================] - 47s 346ms/step - loss: 1.2918 - accuracy: 0.7318 - val_loss: 1.0326 - val_accuracy: 0.6995
Epoch 2/3
125/125 [==============================] - 42s 334ms/step - loss: 0.8543 - accuracy: 0.7632 - val_loss: 0.8669 - val_accuracy: 0.7629
Epoch 3/3
125/125 [==============================] - 42s 338ms/step - loss: 0.7177 - accuracy: 0.8042 - val_loss: 0.7437 - val_accuracy: 0.7933


In [25]:
# During inference
# re-define encoder and decoder model
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_inputs_h = Input(shape=(latent_dimension,))
decoder_state_inputs_c = Input(shape=(latent_dimension,))
decoder_state_inputs = [decoder_state_inputs_h, decoder_state_inputs_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_state_inputs)
decoder_state = [state_h, state_c]
decoder_model = Model(
    [decoder_inputs] + decoder_state_inputs,
    [decoder_outputs] + decoder_state)

In [ ]:
# Reverse look-up
# token index to token / french character
reverse_input_char_index = dict(
    (i,char) for char,i in input_token_index.items())
reverse_input_char_index

In [ ]:
reverse_target_char_index = dict(
    (i,char) for char,i in target_token_index.items())
reverse_target_char_index

In [28]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

